# DLPROF-WIND Plots

[Click here](https://www.arm.gov/capabilities/vaps/dlprof-wind) for more information about this vap.

In [ ]:
%matplotlib widget
import ipywidgets as widgets

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
from datetime import datetime

import act
import xarray as xr

In [ ]:
# Data archive directory
DATA_DIR = r'/data/archive/'

In [ ]:
# Datastream info
DATASTREAM_NAME = 'dlprofwind4news'
DATA_LEVEL = 'c1'
LOCATIONS = [{'end_date': '2017-10-29', 'facility': 'M1', 'site': 'asi', 'start_date': '2016-09-06'}, {'end_date': '2019-04-30', 'facility': 'M1', 'site': 'cor', 'start_date': '2018-09-28'}, {'end_date': '2022-04-16', 'facility': 'C1', 'site': 'ena', 'start_date': '2014-10-21'}, {'end_date': '2015-08-27', 'facility': 'M1', 'site': 'mao', 'start_date': '2014-01-31'}, {'end_date': '2022-04-16', 'facility': 'C1', 'site': 'nsa', 'start_date': '2014-07-30'}, {'end_date': '2021-06-14', 'facility': 'M1', 'site': 'oli', 'start_date': '2014-07-28'}, {'end_date': '2012-03-31', 'facility': 'M1', 'site': 'pgh', 'start_date': '2011-07-21'}, {'end_date': '2013-06-19', 'facility': 'M1', 'site': 'pvc', 'start_date': '2012-06-29'}, {'end_date': '2022-04-17', 'facility': 'C1', 'site': 'sgp', 'start_date': '2010-11-02'}, {'end_date': '2022-04-17', 'facility': 'E32', 'site': 'sgp', 'start_date': '2016-05-03'}, {'end_date': '2022-04-17', 'facility': 'E37', 'site': 'sgp', 'start_date': '2016-05-03'}, {'end_date': '2022-04-16', 'facility': 'E39', 'site': 'sgp', 'start_date': '2016-04-01'}, {'end_date': '2022-04-16', 'facility': 'E41', 'site': 'sgp', 'start_date': '2016-05-03'}, {'end_date': '2015-01-03', 'facility': 'C3', 'site': 'twp', 'start_date': '2010-12-13'}]

## Define site, facility, and date ranges

In [ ]:
print("The following locations and date ranges are available for this VAP:")
display(pd.DataFrame(LOCATIONS, columns=['site', 'facility', 'start_date', 'end_date']))

In [ ]:
# Define site, facility, and date range below:
site_facility = ( 'asi', 'M1' )

# Date range. Format: YYYY-MM-DD
date_start = '2017-10-23'
date_end = '2017-10-28'

## Load data files
Loads data files from /data/archive/

In [ ]:
# Compile list of files
site, facility = site_facility
d_date_start = datetime.strptime(date_start, '%Y-%m-%d')
d_date_end = datetime.strptime(date_end, '%Y-%m-%d')
dir_path = os.path.join(DATA_DIR + site, site + DATASTREAM_NAME + facility + r'.' + DATA_LEVEL )
files_list  = []
for f in os.listdir(dir_path):
    file_date_str = f.split(r'.')[2]
    try:
        file_date = datetime.strptime(file_date_str, '%Y%m%d')
    except Exception:
        continue 
    if d_date_start <= file_date and d_date_end >= file_date:
        files_list.append(os.path.join(dir_path, f))

# Load files as a single dataset
ds = act.io.armfiles.read_netcdf(files_list)
ds.clean.cleanup()
print(f'{len(files_list)} files loaded')
ds

## Plot time series data

In [ ]:
variables_to_plot = ['u', 'v', 'wind_speed']

In [ ]:
ts_display = act.plotting.TimeSeriesDisplay(ds)
ts_display.add_subplots((len(variables_to_plot),), figsize = (15,4*len(variables_to_plot)))

for i,v in enumerate(variables_to_plot):
    ts_ax = ts_display.plot(v, subplot_index=(i,), set_title=ds.variables[v].attrs['long_name'])
    ts_ax.grid()

plt.show()

## Field selection dropdown

In [ ]:
plt.ioff()

available_variables = [v for v in ds.variables if not('time' in v or v.startswith('qc_') or v.startswith('source_')) and 'long_name' in ds.variables[v].attrs]
variable = available_variables[-1]

dropdown = widgets.Dropdown(
    options = [(ds.variables[v].attrs['long_name'], v) for v in available_variables],
    value= variable,
    description='Field:',
    disabled=False,
)

dropdown.layout.margin = '0px 30% 0px 20%'
dropdown.layout.width = '50%'

i_display = act.plotting.TimeSeriesDisplay(ds)
i_display.add_subplots((1,), figsize = (15,5))
i_ax = i_display.plot(variable, subplot_index=(0,), set_title=ds.variables[variable].attrs['long_name'])
i_ax.grid()

i_fig = i_display.fig

def update_plot(change):
    i_ax.cla()
    i_ax_new = i_display.plot(change.new, subplot_index=(0,), set_title=ds.variables[change.new].attrs['long_name'])
    i_ax_new.grid()
    i_fig.canvas.draw()
    i_fig.canvas.flush_events()

dropdown.observe(update_plot, names='value')

widgets.AppLayout(
    header=dropdown,
    center=i_fig.canvas,
    pane_heights=[1, 6,1]
)